<a href="https://colab.research.google.com/github/abiflynn/supervised_machine_learning/blob/main/housing_prices_regression/housing_prices_kaggle_competition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Housing Project: Kaggle Competition

# Import Packages

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_regression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score

# Upgrade Scikit-learn

In [ ]:
# this needs to be run for each new runtime
# because colab has scikit-learn 1.0.2 pre-installed 
# and we need newer version (1.2.0 and higher)
# to use .set_output() method
!pip install scikit-learn --upgrade

# if you plan on running the whole notebook again during the same runtime
# you can comment the line above

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Read the Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = '/content/drive/MyDrive/WBS CODING/Project 7: (Supervised Machine Learning)/Kaggle Competition /train.csv'

housing_df = pd.read_csv(path)
pd.set_option('display.max_columns', None)
housing_df.head(5)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


# X and Y Creation & Data Splitting

In [ ]:
#X and y creation
x = housing_df.drop(columns=['Id'])
y = x.pop("SalePrice")

#Data splitting
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=123)

# Pipeline 

In [ ]:
#Select categorical and numerical column names
x_cat_columns = x.select_dtypes(exclude="number").copy().columns
x_num_columns = x.select_dtypes(include="number").copy().columns

#Create numerical pipeline, only with the SimpleImputer(strategy="mean")
numeric_pipe = make_pipeline(
    SimpleImputer())
 
#Create categorical pipeline, with the SimpleImputer(fill_value="N_A") and the OneHotEncoder
categoric_pipe = make_pipeline(
    SimpleImputer(strategy="constant", fill_value="N_A"),
    OneHotEncoder(sparse_output=False, handle_unknown='ignore')
)

#Created the columntransformer
preprocessor = ColumnTransformer(
    transformers=[
        ("num_pipe", numeric_pipe, x_num_columns), #1 branch called "num_pipe", will apply the steps in the numeric_pipe to the columns named in x_num_columns
        ("cat_pipe", categoric_pipe, x_cat_columns), #2 branch called "cat_pipe", will apply the steps in the categoric_pipe to the columns named in x_cat_columns
    ]
)

#Creating a pipeline 
pipeline = make_pipeline(preprocessor,
                         MinMaxScaler()).set_output(transform='pandas')

#Use the pipeline created above to fit and transform the train data 
x_train_trans_fit = pipeline.fit_transform(x_train, y_train)

#Transform the test set
x_test_trans_fit = pipeline.transform(x_test)

# 1. Decision Tree Regressor

In [ ]:
#Store the decision tree
tree = DecisionTreeRegressor()
#Fit the decicion tree 
tree.fit(x_train_trans_fit, y_train)
#Predict the test data 
tree_pred_dt = tree.predict(X = x_test_trans_fit)

In [ ]:
#Root mean squared error score 
from sklearn.metrics import mean_squared_log_error
rmse_score = mean_squared_log_error(y_test, tree_pred_dt)
rmse_score

0.04347934841875883

In [ ]:
#Create parameter grid
param_grid = {
    "max_depth": range(2, 14),
    "min_samples_leaf": range(3, 10),
    "criterion":['poisson', 'absolute_error', 'friedman_mse', 'squared_error']
}

In [ ]:
#Define cross validation
search = GridSearchCV(tree,
                      param_grid,
                      cv=5,
                      verbose=1, 
                      error_score='raise')

In [ ]:
#Fit the cross validation grid search 
search.fit(x_train_trans_fit, y_train)


Fitting 5 folds for each of 336 candidates, totalling 1680 fits


GridSearchCV(cv=5, error_score='raise', estimator=DecisionTreeRegressor(),
             param_grid={'criterion': ['poisson', 'absolute_error',
                                       'friedman_mse', 'squared_error'],
                         'max_depth': range(2, 14),
                         'min_samples_leaf': range(3, 10)},
             verbose=1)

In [ ]:
#Predict the new data 
search_predict = search.predict(X = x_test_trans_fit)

In [ ]:
#Root mean squared error score 
from sklearn.metrics import mean_squared_log_error
rmse_score = mean_squared_log_error(y_test, search_predict)
rmse_score

0.0341148665543142

# 2. K-Best

In [ ]:
from sklearn.feature_selection import SelectKBest, f_regression

In [ ]:
#Creating a pipeline #Initialize KBest, using the f-regression and setting k=10.
pipeline_kbest = make_pipeline(preprocessor,
                               MinMaxScaler(),
                               SelectKBest(score_func=f_regression, k=10)).set_output(transform='pandas')


In [ ]:
#Use the pipeline created above to fit and transform the train data 
x_train_kbest = pipeline_kbest.fit_transform(x_train, y_train)

#Transform the test set
x_test_kbest = pipeline_kbest.transform(x_test)

In [ ]:
#Store the decision tree
tree = DecisionTreeRegressor()
#Fit the decicion tree 
tree.fit(x_train_kbest, y_train)
#Predict the test data 
tree_pred_kbest = tree.predict(X = x_test_kbest)

In [ ]:
#Root mean squared error score 
from sklearn.metrics import mean_squared_log_error
rmse_score = mean_squared_log_error(y_test, tree_pred_kbest)
rmse_score

0.04435215706459573

In [ ]:
#Store the K-Nearest Neighbor
neigh = KNeighborsRegressor(n_neighbors=12)
#Fit 
neigh.fit(x_train_kbest, y_train)
#Predict the test data 
knn_pred_kbest = neigh.predict(X = x_test_kbest)

In [ ]:
#Root mean squared error score 
from sklearn.metrics import mean_squared_log_error
rmse_score = mean_squared_log_error(y_test, knn_pred_kbest)
rmse_score

0.02742593074951493

In [ ]:
#Logistic regression
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(x_train_kbest, y_train)
lr_pred_kbest = lr.predict(X = x_test_kbest)

In [ ]:
#Root mean squared error score 
from sklearn.metrics import mean_squared_log_error
rmse_score = mean_squared_log_error(y_test, lr_pred_kbest)
rmse_score

0.06791813547626303

In [ ]:
#Random forest 
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=100)
rf.fit(X = x_train_kbest, y = y_train)
rf_pred_kbest = rf.predict(X = x_test_kbest)

In [ ]:
#Root mean squared error score 
from sklearn.metrics import mean_squared_log_error
rmse_score = mean_squared_log_error(y_test, rf_pred_kbest)
rmse_score

0.022482336833612185

In [ ]:
#Calling in the test data 
path ='/content/drive/MyDrive/WBS CODING/Project 7: (Supervised Machine Learning)/Kaggle Competition /test.csv'

final_data = pd.read_csv(path)

In [ ]:
#Transform the final data
final_data_trans = pipeline_kbest.transform(final_data)

#Create the correct format for the final data 
final_data_test = rf.predict(final_data_trans)
final_data_test_1 = pd.DataFrame(final_data_test, columns=["SalePrice"])
final_data_test_1.index = final_data_test_1.index +1461
final_data_test_1.index.rename("Id", inplace=True)
final_data_test_1

,SalePrice
Id,
1461,126524.000000
1462,153275.000000
1463,173618.350000
1464,182835.000000
1465,223766.500000
...,...
2915,82700.333333
2916,82485.285714
2917,130422.000000


In [ ]:
#Export 
final_data_test_1.to_csv(path+"kaggle_submission_1_random_forest.csv")

# Random Forest 

In [ ]:
#Creating a pipeline #Initialize KBest, using the f-regression and setting k=10.
pipeline_kbest_rf = make_pipeline(preprocessor,
                                  MinMaxScaler(),
                                  SelectKBest(score_func=f_regression, k=10)).set_output(transform='pandas')

In [ ]:
#Use the pipeline created above to fit and transform the train data 
x_train_kbest_rf = pipeline_kbest_rf.fit_transform(x_train, y_train)

#Transform the test set
x_test_kbest_rf = pipeline_kbest_rf.transform(x_test)

In [ ]:
#Random forest 
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=100)

rf.fit(X = x_train_kbest, y = y_train)
rf_pred_kbest = rf.predict(X = x_test_kbest)

In [ ]:
#Create parameter grid
param_grid = {
    "n_estimators": range(90,105),
    "max_features": ["auto", "sqrt"], 
    "max_depth":range(14,16),
}

In [ ]:
from sklearn.model_selection import GridSearchCV
#Define cross validation
search_rf = GridSearchCV(rf,
                      param_grid,
                      cv=5,
                      verbose=1, 
                      error_score='raise')

In [ ]:
search_rf.fit(X = x_train_kbest_rf, y = y_train)

In [ ]:
search_rf.best_params_

{'max_depth': 14, 'max_features': 'sqrt', 'n_estimators': 92}

In [ ]:
serach_predict_rf = search_rf.predict(X = x_test_kbest)

#Root mean squared error score 
from sklearn.metrics import mean_squared_log_error
rmse_score = mean_squared_log_error(y_test, serach_predict_rf)
rmse_score

0.021661741535842164

In [ ]:
#Calling in the test data 
path ='/content/drive/MyDrive/WBS CODING/Project 7: (Supervised Machine Learning)/Kaggle Competition /test.csv'

final_data = pd.read_csv(path)

In [ ]:
#Transform the final data
final_data_trans = pipeline_kbest_rf.transform(final_data)

#Create the correct format for the final data 
final_data_test = search_rf.predict(final_data_trans)
final_data_test_1 = pd.DataFrame(final_data_test, columns=["SalePrice"])
final_data_test_1.index = final_data_test_1.index +1461
final_data_test_1.index.rename("Id", inplace=True)
final_data_test_1

,SalePrice
Id,
1461,122610.245201
1462,157579.195804
1463,176072.251697
1464,187440.497043
1465,211630.496429
...,...
2915,84633.479202
2916,82543.684516
2917,135460.839431


In [ ]:
#Export 
final_data_test_1.to_csv(path+"kaggle_submission_2_random_forest.csv")

# SGDRegressor

In [ ]:
#Creating a pipeline #Initialize KBest, using the f-regression and setting k=10.
pipeline_kbest_sgd = make_pipeline(preprocessor,
                                  MinMaxScaler()).set_output(transform='pandas')
                                  #SelectKBest(score_func=f_regression, k=10)).set_output(transform='pandas')

In [ ]:
#Use the pipeline created above to fit and transform the train data 
x_train_kbest_sgd = pipeline_kbest_sgd.fit_transform(x_train, y_train)

#Transform the test set
x_test_kbest_sgd = pipeline_kbest_sgd.transform(x_test)

In [ ]:
#SGDRegressor 
from sklearn.linear_model import SGDRegressor

sgd = SGDRegressor()

sgd.fit(X = x_train_kbest_sgd, y = y_train)
sgd_pred_kbest = sgd.predict(X = x_test_kbest_sgd)

In [ ]:
serach_predict_sgd = sgd.predict(X = x_test_kbest_sgd)

#Root mean squared error score 
from sklearn.metrics import mean_squared_log_error
rmse_score = mean_squared_log_error(y_test, serach_predict_sgd)
rmse_score

0.016599915643460004

In [ ]:
#Calling in the test data 
path ='/content/drive/MyDrive/WBS CODING/Project 7: (Supervised Machine Learning)/Kaggle Competition /test.csv'

final_data = pd.read_csv(path)

In [ ]:
#Transform the final data
final_data_trans = pipeline_kbest_sgd.transform(final_data)

#Create the correct format for the final data 
final_data_test = sgd.predict(final_data_trans)
final_data_test_1 = pd.DataFrame(final_data_test, columns=["SalePrice"])
final_data_test_1.index = final_data_test_1.index +1461
final_data_test_1.index.rename("Id", inplace=True)
final_data_test_1

,SalePrice
Id,
1461,112743.836508
1462,145503.459708
1463,177287.421379
1464,183591.593718
1465,192684.001907
...,...
2915,69857.483352
2916,76991.997058
2917,159948.356340


In [ ]:
#Export 
final_data_test_1.to_csv(path+"kaggle_submission_3_sgd.csv")